In [ ]:
!pip install rasterio numpy

In [3]:
import rasterio as rast
import numpy as np

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 0.0, 'width': 51218, 'height': 89289, 'count': 1, 'crs': CRS.from_epsg(32618), 'transform': Affine(10.0, 0.0, 243910.0,
       0.0, -10.0, 5320650.0)}
(1, 89289, 51218)


In [4]:
obj22 = rast.open('data/2022.tif')
obj23 = rast.open('data/2023.tif')
obj24 = rast.open('data/2024.tif')

In [24]:
obj22.transform * (0,0)

(243910.0, 5320650.0)

In [25]:
obj23.transform * (obj22.width, obj22.height)

(756090.0, 4427760.0)

In [27]:
meta = obj22.meta
meta.update(count=3)

with rast.open('data/combined.tif', 'w', **meta) as dst:
    dst.write(obj22.read(1), 1)
    dst.write(obj23.read(1), 2)
    dst.write(obj24.read(1), 3)

In [66]:
obj = rast.open('data/combined.tif')

In [31]:
obj.read(1)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [32]:
obj.crs

CRS.from_epsg(32618)

In [67]:
nyc_bounds = {
    "west": -74.25909,
    "east": -73.70018,
    "north": 40.91553,
    "south": 40.49612
}

In [38]:
!pip3.12 install pyproj

   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   - -------------------------------------- 0.2/6.1 MB 2.6 MB/s eta 0:00:03
   ----- ---------------------------------- 0.8/6.1 MB 7.0 MB/s eta 0:00:01
   ------- -------------------------------- 1.1/6.1 MB 7.3 MB/s eta 0:00:01
   ------------ --------------------------- 2.0/6.1 MB 9.6 MB/s eta 0:00:01
   ----------------- ---------------------- 2.6/6.1 MB 10.5 MB/s eta 0:00:01
   ----------------- ---------------------- 2.7/6.1 MB 9.1 MB/s eta 0:00:01
   ----------------- ---------------------- 2.7/6.1 MB 9.1 MB/s eta 0:00:01
   ------------------- -------------------- 2.9/6.1 MB 7.5 MB/s eta 0:00:01
   -------------------------- ------------- 4.0/6.1 MB 9.2 MB/s eta 0:00:01
   ------------------------------ --------- 4.6/6.1 MB 9.6 MB/s eta 0:00:01
   ----------------------------------- ---- 5.4/6.1 MB 10.2 MB/s eta 0:00:01
   ---------------------


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [68]:
from pyproj import Transformer
# Initialize the transformer to convert from lat/lon to the specified CRS
transformer = Transformer.from_crs("epsg:4326", "epsg:32618")

# Transform the bounding box to the CRS of the raster
west, north = transformer.transform(nyc_bounds["north"], nyc_bounds["west"])
east, south = transformer.transform(nyc_bounds["south"], nyc_bounds["east"])

In [64]:
# Open the raster file and clip it to the bounding box
window = rast.windows.from_bounds(west, south, east, north, obj.transform)
out_image = obj.read(window=window)
out_transform = obj.window_transform(window)
out_meta = obj.meta.copy()

WindowError: Bounds and transform are inconsistent

In [43]:
# Update the metadata to reflect the new dimensions, transform, and CRS
out_meta.update({
    "driver": "GTiff",
    "height": out_image.shape[1],
    "width": out_image.shape[2],
    "transform": out_transform
})

In [44]:
with rast.open('nyc.tif', 'w', **out_meta) as dst:
    dst.write(out_image)

In [45]:
nyc = rast.open("data/nyc.tif")

In [46]:
nyc.indexes

(1, 2, 3)

In [50]:
obj.transform*(obj.width, obj.height), nyc.transform*(nyc.width, nyc.height)

((756090.0, 4427760.0), (610151.3996412894, 4483634.373562211))

In [51]:
obj.bounds, nyc.bounds

(BoundingBox(left=243910.0, bottom=4427760.0, right=756090.0, top=5320650.0),
 BoundingBox(left=562391.3996412894, bottom=4483634.373562211, right=610151.3996412894, top=4529644.373562211))

In [52]:
obj.transform * (0,0), nyc.transform * (0,0)

((243910.0, 5320650.0), (562391.3996412894, 4529644.373562211))